# Statistical Learning Midterm: Predicting Hurricane Impact Risk
## Equitable Disaster Relief through Machine Learning

**Date:** October 7, 2025  
**Context:** AURA MVP - Hurricane disaster assistance prediction model

---
## 1. Problem Statement & Motivation

### The Challenge
After major hurricanes, **how much disaster assistance will each community need?**

**Why it matters:**
- FEMA needs to pre-position resources before storms
- Vulnerable communities often under-apply for aid due to barriers
- Predicting need (not just claims) can reveal equity gaps

### Research Question
**Can we predict tract-level FEMA claims using demographic, housing, and disaster data to identify communities at risk of being underserved?**

---
## 2. Data Sources & Pipeline

### Data Sources
| Source | Features | Purpose |
|--------|----------|--------|
| **FEMA Individual Assistance** | Total claims, applications, eligibility rates | Outcome variable |
| **CDC Social Vulnerability Index** | Poverty, elderly, language, race/ethnicity | Vulnerability indicators |
| **US Census (ACS 2022)** | Population, income, housing units | Demographic context |
| **NOAA HURDAT2** | Hurricane tracks, wind, rainfall | Physical hazard (future work) |

### Dataset Overview
- **Unit of analysis:** Census tract × Storm event
- **Geographic scope:** TX, LA, MS, AL, FL (Gulf Coast)
- **Storms:** Harvey, Irma, Michael, Laura, Ida, Ian (2017-2022)
- **Total observations:** 5,668 tract-storm pairs
- **Tracts with claims:** 1,557 (27.5%)

In [ ]:
# Load and preview data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

# Load tract_storm dataset
df = pd.read_csv("../../data/processed/tract_storm_features.csv")

print(f"Dataset shape: {df.shape}")
print(f"\nTarget variable (fema_claims_total) summary:")
print(df['fema_claims_total'].describe())

# Show storm distribution
storm_names = {
    4332: "Harvey (2017)",
    4337: "Irma (2017)",
    4399: "Michael (2018)",
    4559: "Laura (2020)",
    4611: "Ida (2021)",
    4673: "Ian (2022)"
}

storm_counts = df['disasterNumber'].value_counts().sort_index()
print(f"\nStorm distribution:")
for disaster_num, count in storm_counts.items():
    storm_name = storm_names.get(disaster_num, f"Unknown ({disaster_num})")
    print(f"  {storm_name}: {count} tracts")

---
## 3. Modeling Approach

### Target Variable
**`fema_claims_total`**: Total FEMA personal property verified losses (in dollars)
- Log-transformed for training: `log1p(claims)` to handle skewed distribution
- Evaluated on raw scale for interpretability

### Algorithms Tested
1. **K-Nearest Neighbors (KNN)** - Distance-based, non-parametric
2. **Decision Tree** - Interpretable, hierarchical splits
3. **Random Forest** - Ensemble of trees, reduced overfitting
4. **Bagging** - Bootstrap aggregation with decision tree base

### Training Strategy
- **Train/Test split:** 80/20 (stratified random)
- **Cross-validation:** 5-fold CV for hyperparameter tuning
- **Preprocessing:** Median imputation + StandardScaler
- **Scoring metric:** Negative RMSE (for grid search)

### Hyperparameter Grids

**KNN:**
- n_neighbors: [5, 10, 15, 25, 35]
- weights: ['uniform', 'distance']
- p: [1, 2] (Manhattan vs Euclidean)

**Decision Tree:**
- max_depth: [None, 5, 10, 20]
- min_samples_split: [2, 5, 10]
- min_samples_leaf: [1, 2, 5]

**Random Forest:**
- n_estimators: [200, 400]
- max_depth: [None, 10, 20]
- min_samples_split: [2, 5]
- max_features: ['sqrt', 0.75, 1.0]

**Bagging:**
- n_estimators: [100, 200, 400]
- max_samples: [0.5, 0.75, 1.0]
- max_features: [0.5, 0.75, 1.0]

---
## 4. Results

### Performance Metrics Table

In [ ]:
# Load performance results
results = pd.read_csv("../../docs/tract_storm_model_performance.csv")

# Format for display
results_display = results.copy()
results_display['model'] = results_display['model'].map({
    'knn': 'KNN',
    'decision_tree': 'Decision Tree',
    'random_forest': 'Random Forest',
    'bagging': 'Bagging'
})
results_display['phase'] = results_display['phase'].str.capitalize()

# Round metrics
for col in ['r2', 'rmse', 'mae', 'mape', 'explained_variance']:
    results_display[col] = results_display[col].round(4)

results_display

### Key Findings

**Best Model: Bagging Regressor**
- **R² = 0.93** (explains 93% of variance)
- **RMSE = $980** (typical prediction error)
- **MAE = $262** (average absolute error)
- **MAPE = 27%** (percentage error)

**Observations:**
1. **Ensemble methods dominate:** Random Forest & Bagging both achieve R² > 0.92
2. **Decision Tree improved significantly** with tuning (0.81 → 0.84 R²)
3. **KNN performance degraded** with tuning (0.79 → 0.70 R²) - suggests optimal parameters were in baseline
4. **Tuning benefits vary** - not always better, but ensemble methods are robust

### Visualizations

In [ ]:
# Display performance comparison
from IPython.display import Image

Image(filename="../visuals/model_results/01_performance_comparison.png", width=900)

In [ ]:
# Actual vs Predicted
Image(filename="../visuals/model_results/02_actual_vs_predicted.png", width=900)

**Interpretation:**
- Strong linear correlation between actual and predicted values
- Some underprediction of extreme values (high-claim tracts)
- Bagging and Random Forest show tightest clustering around perfect prediction line

In [ ]:
# Feature Importance
Image(filename="../visuals/model_results/04_feature_importance.png", width=900)

**Top Predictive Features:**
- **Applications** - Number of FEMA applications (strongest predictor)
- **Housing units** - Total housing in tract
- **Demographic vulnerability** - Poverty rate, elderly population
- **Income** - Median household income
- **Race/ethnicity** - Hispanic and Black population percentages

**Key Insight:** The model learns heavily from **application behavior** and **population size**, not just physical risk!

---
## 5. Responsible AI: Bias Analysis

### The Equity Question
**Does the model systematically underpredict for vulnerable communities?**

If yes → Model could mask true need and perpetuate underservice

In [ ]:
# Residuals by vulnerability quartiles
Image(filename="../visuals/model_results/03_residuals_by_vulnerability.png", width=900)

### Bias Diagnostic Results

**Analysis Method:**
- Segment test set into quartiles by vulnerability indicators
- Compare residuals (actual - predicted) across quartiles
- Negative residuals = underprediction (model predicts too low)

**Findings:**
1. **Poverty:** Slight underprediction in Q4 (highest poverty) for all models
2. **Hispanic %:** Relatively balanced across quartiles
3. **Black %:** Some underprediction in Q4 for Bagging/Random Forest
4. **Limited English:** Minimal systematic bias

**Interpretation:**
- Models show **moderate equity concerns** - high-poverty and high-Black tracts are slightly underestimated
- Residuals center near zero (good), but variance increases in vulnerable quartiles
- **Root cause:** FEMA claims data reflects application behavior, not true damage

### Data Limitations & Bias Sources

**Problem: The target variable is biased**
- **FEMA claims ≠ actual damage**
- Barriers to application:
  - Language access
  - Digital literacy
  - Trust in government
  - Awareness of eligibility
  
**Impact on model:**
- Model learns "who applies" not "who needs help"
- Vulnerable communities with low claims may have high unmet need

**Missing critical data:**
- Physical hazard features (wind speed, rainfall, flood zones) are placeholders
- Without them, model relies on demographic proxies
- High-risk Black/Hispanic tracts may be undercounted if they didn't apply

---
## 6. Responsible AI: Safeguards & Recommendations

### Policy Interpretation Guidance

**What this model predicts:**
- Expected FEMA claims **given historical application patterns**
- NOT eligibility, NOT true damage, NOT need

**How to use predictions ethically:**
1. **Flag discrepancies:** Low predicted claims + high vulnerability = investigate for barriers
2. **Prioritize outreach:** High-poverty, low-application tracts need proactive assistance
3. **Combine with qualitative data:** Partner with local VOADs and community orgs
4. **Human-in-the-loop:** Never auto-deny aid based on model predictions

### Technical Safeguards

**Implemented:**
1. ✅ Bias diagnostics by vulnerability quartiles
2. ✅ Feature importance analysis (transparency)
3. ✅ Multiple evaluation metrics (not just R²)

**Recommended next steps:**
1. **Add physical hazard features** (NOAA wind/rain) to decouple risk from demographics
2. **Counterfactual stress tests:** Swap demographics while holding hazard constant
3. **Fairness constraints:** Penalize models that underpredict for vulnerable groups
4. **Threshold transparency:** Publish cut-points and expected false negative rates

### Who Benefits? Who Gets Harmed?

**Benefits:**
- **FEMA/Emergency managers:** Better resource pre-positioning
- **High-application communities:** Model captures their patterns well
- **Researchers:** Framework for equity-aware disaster modeling

**Potential harms:**
- **Low-application vulnerable communities:** Risk being deprioritized
- **Non-English speakers:** Language barriers invisible to model
- **Mobile home residents:** May not know eligibility, underrepresented in claims

**Mitigation:**
- Use predictions to **increase aid**, not ration it
- Deploy proactive outreach to low-prediction/high-vulnerability tracts
- Combine with ground truth from local partners

---
## 7. Hurricane Physical Features (Future Work)

### NOAA HURDAT2 Integration

**Features to add:**
- `tract_distance_km`: Distance from tract centroid to storm path
- `max_wind_mph`: Maximum wind speed experienced
- `exposure_duration_hours`: Hours above wind thresholds
- `pct_floodplain`: Tract area in FEMA flood zones

**Expected impact:**
- Shift importance from demographics to physical risk
- Reveal where demographic patterns are proxies for hazard exposure
- Improve predictions for future storms without historical claims data

### Hurricane Ida Example Visualization

In [ ]:
# Note: These visualizations are from collaborator's NOAA pipeline
# Showing methodology for physical feature extraction

from IPython.display import IFrame

print("Interactive map: Hurricane Ida wind field coverage")
print("Opens in: ../visuals/hurricane_features/ida_interactive_map.html")
print("\nShows: Census tract centroids, storm track, wind radii envelopes")

---
## 8. Conclusions

### Summary of Findings

**Model Performance:**
- Achieved **R² = 0.93** with Bagging ensemble
- All 4 required algorithms successfully implemented and tuned
- Ensemble methods (Random Forest, Bagging) significantly outperform single models

**Equity Insights:**
- Moderate bias detected: High-poverty and high-Black tracts slightly underpredicted
- Root cause: Target variable (FEMA claims) reflects application behavior, not need
- Model learns "who applies" rather than "who needs help"

**Key Limitation:**
- Missing physical hazard features (wind, rain, flood zones)
- Without them, model relies on demographic proxies
- Strong performance suggests demographics/housing correlate with risk, but also with application barriers

### Policy Implications

**For disaster response:**
1. Use predictions to **identify under-application**, not allocate resources
2. Flag low-prediction + high-vulnerability tracts for proactive outreach
3. Combine ML predictions with community partnerships

**For AURA MVP:**
- Framework demonstrates feasibility of tract-level risk prediction
- Next phase: Integrate NOAA physical features
- Pilot with 2-3 Gulf Coast states before national rollout

### Future Enhancements

**Immediate (< 1 month):**
1. Integrate NOAA wind/distance features from collaborator pipeline
2. Add FEMA flood zone overlays
3. Rerun models and compare feature importance shifts

**Short-term (1-3 months):**
1. Fairness-aware tuning (penalize underprediction of vulnerable tracts)
2. Temporal validation (train on 2017-2020, test on 2021-2022)
3. State-specific models to capture regional policy differences

**Long-term (3-6 months):**
1. Real-time prediction API for pre-storm resource allocation
2. Dashboard for emergency managers with equity indicators
3. Integration with CDC SVI updates and FEMA declaration polygons

---
## Appendix: Technical Details

### Best Hyperparameters

In [ ]:
import json

with open("../../docs/tract_storm_best_params.json", "r") as f:
    best_params = json.load(f)

for model, params in best_params.items():
    print(f"\n{model.upper()}:")
    for param, value in params.items():
        print(f"  {param}: {value}")

### Feature List

In [ ]:
# Show all features used in modeling
exclude_cols = {
    "fema_claims_total_log1p", "fema_claims_total", "fema_claims_pc",
    "population_for_pc", "tract_geoid", "disasterNumber", "state_abbr",
    "county_name", "acs_state_fips", "state_fips", "coverage_label"
}

feature_cols = [col for col in df.columns if col not in exclude_cols]
numeric_features = df[feature_cols].select_dtypes(include=[np.number]).columns.tolist()

print(f"Total features: {len(numeric_features)}\n")
print("Features by category:\n")

categories = {
    "FEMA Access": ['applications', 'declaration', 'tsa_eligible_rate', 'owner_rate', 'renter_rate', 'fema_claims_mean'],
    "Demographics": ['total_population', 'median_household_income', 'pct_poverty', 'pct_elderly', 'pct_children'],
    "Race/Ethnicity": ['pct_black', 'pct_hispanic', 'pct_limited_english'],
    "Housing": ['housing_units', 'pct_mobile_homes', 'pct_multi_unit', 'pct_crowded_housing', 'pct_no_vehicle'],
    "Hazard (Placeholder)": ['max_wind_mph', 'total_rainfall_in', 'min_distance_km', 'pct_floodplain'],
    "Geography": ['area_sq_mi', 'in_fema_decl', 'in_noaa_exposure']
}

for category, features in categories.items():
    available = [f for f in features if f in numeric_features]
    print(f"{category}: {len(available)} features")
    for f in available:
        print(f"  - {f}")
    print()

### Data Quality Notes

**Missing data handling:**
- Median imputation for continuous features
- Hazard features (wind, rain, distance, floodplain) are entirely missing - awaiting NOAA integration
- Demographic features have <5% missingness

**Target variable transformation:**
- Original: Highly right-skewed (many zeros, few very large values)
- Transformed: log1p(fema_claims_total) for training
- Evaluation: Back-transformed to raw dollars for interpretability

**Cross-validation strategy:**
- 5-fold CV with random splits (not stratified by storm)
- Each fold: 80% train, 20% validation
- Final test set: 20% held out from all training

---
## References

**Data Sources:**
- FEMA Individual Assistance Program: https://www.fema.gov/openfema-data-page/individuals-and-households-program-valid-registrations-v1
- CDC Social Vulnerability Index 2022: https://www.atsdr.cdc.gov/placeandhealth/svi/data_documentation_download.html
- US Census American Community Survey: https://www.census.gov/programs-surveys/acs
- NOAA HURDAT2 Hurricane Database: https://www.nhc.noaa.gov/data/hurdat/

**Collaborator:**
- Hurricane physical feature extraction pipeline: https://github.com/mhahn2011/HURRICANE-DATA-ETL

**AURA Project Context:**
- Goal: Equitable disaster assistance through ML-powered resource allocation
- Partners: FEMA, state emergency management, VOADs
- Geographic focus: Gulf Coast states (hurricane-prone regions)